**Author:** Shahab Fatemi

**Email:** shahab.fatemi@umu.se   ;   shahab.fatemi@amitiscode.com

**Created:** 2024-11-xx

**Last update:** 2025-09-11

**MIT License** — Shahab Fatemi (2025); For use in the *Machine Learning in Physics* course, Umeå University, Sweden; See the full license text in the parent folder.

<hr>

📢 <span style="color:red"><strong> Note for Students:</strong></span>

* Before working on the labs, review your lecture notes.

* Please read all sections, code blocks, and comments **carefully** to fully understand the material. Throughout the labs, my instructions are provided to you in written form, guiding you through the materials step-by-step.

* All concepts covered in this lab are part of the course and may be included in the final exam.

* I strongly encourage you to work in pairs and discuss your findings, observations, and reasoning with each other.

* If something is unclear, don't hesitate to ask.

* Exercise submission is not required; these tasks are designed to help you practice, explore the concepts, and learn by doing.

* I have done my best to make the lab files as bug-free (and error-free) as possible, but remember: *there is no such thing as bug-free code.* If you observed any bugs, errors, typos, or other issues, I would greatly appreciate it if you report them to me by email. Verbal notifications are not work, as I will likely forget 🙂

ENJOY WORKING ON THIS LAB.
***

# 🛠️ Purpose and Learning Outcomes:

In this lab, you will write your own Gradient Descent (GD) algorithm and try it out on different datasets. We will start with the basic version, the Batch GD, and then take a look at how Stochastic GD (SGD) works. You will also see why feature scaling matters by testing an example where the SGD algorithm struggles without scaling.

At the end, there is an optional challenge: try using Gradient Descent to solve a physics problem that has nothing to do with machine learning but one can uses GD to find the solution, emphasizing on the importance and global applications of GD.

***

### Before we begin

In all previous notebooks we worked on, we needed to explicitly set the figure size, axis font size, and other parameters to ensure consistent and clear visualizations. To streamline this process, I've created a utility function that automatically configures these settings for you. This function is located in the `./utils/notebook_config.py` file within the parent directory. When these settings are defined in the `notebook_config.py` file, you can simply call the function at the beginning of your notebook to apply them throughout your visualizations. In general, it's a good practice to encapsulate such configurations in a utility function to promote code *reusability* and *maintainability*.
You can modify the utility function to customize the default settings according to your preferences, and add or remove any configurations as needed.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../utils'))
from notebook_config import *

# Gradient Descent (GD)

**Overview:** Gradient Descent (GD) is an optimization algorithm used to minimize the cost function. The goal is to find the best-fitting line (or hyperplane) that predicts the output variable based on input features.

GD follows these steps:
1. *Initialize*: Start with an initial guess for model weights (parameters or coefficients).
2. *Compute predictions*: Use current parameters to calculate predicted values.
3. *Calculate cost*: Compute the cost using MSE.
4. *Compute gradients*: Calculate gradients of the cost function with respect to the parameters.
5. *Update parameters*: Adjust parameters in the opposite direction of the gradient using a small learning rate.
6. *Iterate*: Repeat until the solution converges or a set number of iterations is reached.

In Matrix form for the entire batch
$
\mathbf{w}_{new} = \mathbf{w}_{old} - \alpha \frac{2}{n} \mathbf{x}^\top (\mathbf{x} \mathbf{w}_{old} - \mathbf{y})
$


***
## ⛷️ Exercise

In this section, you need to write your own code and apply the concepts you have learned so far by working on a small dataset stored in `../datasets/simple_grad_descent_data1.csv`. Start by loading the dataset from the provided path using pandas. Once the data is loaded, split it into a training and validation sets. 

After than, analyze and visualize the training set to get an initial sense of the data distribution. Use scatter plots or other relevant visualizations to understand how the input features relate to the target variable.

Based on the overview text I provided above and using your lecture notes, develop your own Batch Gradient Descent algorithm for linear regression and track the cost over iterations.

Once your implementation is complete, run it on normalized data (later you will see why this is important), and evaluate your model using all relevant metrics; e.g., compute the training error, validation error, visualize the cost function convergence, learning curve, etc. Discuss whether your model appears to underfit, overfit, or generalize well. Use the results of your error metrics and plots to justify your conclusions. Do not spend too much time on the graphics or animating the results. Your focus should be on the correctness of your algorithm implementation, and data analysis.

Explore how the learning rate (`alpha`) influence the convergence of the gradient descent algorithm?

Now that you made sure your model is implemented correctly, you can apply it to a new dataset stored in `../datasets/simple_grad_descent_data2.csv`. 

At the very end of this notebook, I've provided the true functions for the developed datasets.
***

## SGD in sklearn and the effect of Normalization

In this section, we use `SGDRegressor` from Scikit-Learn to solve a regression problem using Stochastic Gradient Descent (SGD). First, review your notes on batch, stochastic and mini-batch GD. We build a pipeline that includes optional normalization and polynomial feature expansion.

Since SGD is sensitive to feature scaling, we compare runs with and without normalization. You will see that normalization not only improves convergence speed but also reduces sensitivity to the learning rate.

Before that, let's generate a simple data including 2 features, x1 and x2, and visualize.

In [ ]:
def generate_data(num_samples=100, noise_level=5):
    np.random.seed(42)
    x1 = np.random.uniform(1, 2, num_samples)
    x2 = np.random.uniform(1, 2000, num_samples)
    y = 4 + 2*x1 - 5*x2
    y += np.random.normal(0, noise_level, size=num_samples)  # add some noise
    x = np.c_[x1, x2]
    return x, y

In [ ]:
from sklearn.model_selection import train_test_split

# Generate and split data
x, y = generate_data(num_samples=200, noise_level=2)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=0)

# Plot training set
plt.figure(figsize=(5, 4))
plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train, cmap="viridis", edgecolor="k", alpha=0.7)
plt.colorbar(label="y")
plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Training data")
plt.grid(True)
plt.show()

Before moving forward, review the `SGDRegressor` manual from:
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor

# Run SGD with optional normalization
def run_sgd(x, y, normalize, learning_rate, max_iter):
    if normalize:
        model = make_pipeline( StandardScaler(), # normalize features
                SGDRegressor(learning_rate='constant', # force constant learning rate to make results comparable
                            eta0=learning_rate, 
                            max_iter=max_iter, 
                            tol=1e-9, 
                            random_state=42) )
    else:
        model = SGDRegressor(learning_rate='constant', # force constant learning rate to make results comparable
                             eta0=learning_rate, 
                             max_iter=max_iter, 
                             tol=1e-9, 
                             random_state=42)
    # Fit model
    model.fit(x, y)

    if normalize:
        # Extract weights and bias from the pipeline
        # and transform them to original scale
        sgdreg  = model.named_steps['sgdregressor']
        scaler  = model.named_steps['standardscaler']
        scale   = scaler.scale_
        mean    = scaler.mean_
        weights = sgdreg.coef_ / scale
        bias    = sgdreg.intercept_ - np.dot(sgdreg.coef_, mean / scale)
    else:
        sgdreg  = model
        weights = sgdreg.coef_
        bias    = sgdreg.intercept_

    return bias, weights

# Run without normalization
bias_no, weights_no = run_sgd(x_train, y_train, normalize=False, learning_rate=1.0e-4, max_iter=10000)
print("No normalization -> bias, w:", bias_no, weights_no)

# Run with normalization
bias_norm, weights_norm = run_sgd(x_train, y_train, normalize=True, learning_rate=1.0e-4, max_iter=10000)
print("With normalization -> bias, w:", bias_norm, weights_norm)

***
### 💡 Reflect and Run

- What do you observe? Can you change the SGDRegressor hyper-parameters such that the weights without normalization get close to those with normalization?

***

## Example: Finding the ground state of a quantum system

If you still have time, let's solve a physics-related optimization problem together. This is not directly related to the Machine Learning, but uses the Gradient Descent method as an optimizer. In case you found errors in my equations and calculations, please correct them and let me know.

The ground state, the state of a system with the lowest possible energy allowed by quantum mechanics, corresponds to the eigenstate of the Hamiltonian operator ($\hat{H}$) with the lowest eigenvalue (energy) such that 
$$
\hat{H}\psi_0​ = E_0 \psi_0
$$

The trial wave function, representing a Gaussian wavefunction, is defined as
$$
\psi(x,\alpha)=e^{−\alpha x^2}
$$

The Hamiltonian operator ($\hat{H}$) is defined as $\hat{T} + \hat{V}$, where $\hat{T}$ is the kinetic energy operator, and $\hat{V}$ is the potential energy operator (harmonic oscillator potential).

The second derivative of the wavefunction is approximated using the finite-difference method:
$$
\frac{\partial^2 \psi(x)}{\partial x^2} \approx \frac{\psi(x + \Delta x) - 2\psi(x) + \psi(x - \Delta x)}{\Delta x^2}
$$

Assuming $\hbar=m=1$, then

$$
T = -\frac{1}{2} \frac{\partial^2 \psi(x)}{\partial x^2} \\
V = \frac{1}{2} \omega^2 x^2 \psi(x)
$$

The total Hamiltonian acting on the wavefunction becomes $\hat{H}\psi(x)=T(x)+V(x)$, and the expectation value of the Hamiltonian in discretized form (with normalization included) is
$$
\langle H \rangle = \frac{\sum_i \psi^*(x_i) \hat{H} \psi(x_i) \Delta x}{\sum_i |\psi(x_i)|^2 \Delta x}
$$

To optimize $\alpha$ (the variational parameter), we can minimize energy using gradient descent. The gradient of the energy with respect to $\alpha$ is approximated using the central difference method,
$$
\frac{\partial \langle H \rangle}{\partial \alpha} \approx \frac{\langle H \rangle(\alpha + \delta \alpha) - \langle H \rangle(\alpha - \delta \alpha)}{2 \delta \alpha}
$$

The variational parameter $\alpha$ is updated iteratively using the gradient descent algorithm. The update rule is given by:
$$
\alpha_{n+1} = \alpha_n - \eta \cdot \frac{\partial \langle H \rangle}{\partial \alpha}.
$$

In [ ]:
# Constants (in atomic units)
m     = 1.0     # Mass
omega = 1.0     # Angular frequency
dx    = 0.1     # Discretization step size
x_min = -1.0    # Min range of x
x_max = +1.0    # Max range of x
num_points = int((x_max - x_min) / dx)      # Number of grid points
x = np.linspace(x_min, x_max, num_points)  # Spatial grid

# Trial wavefunction (Gaussian form)
# x is the position variable
# alpha is the variational parameter for the wavefunction
def wavefunction(x, alpha):
    return np.exp(-alpha * x**2)

# Hamiltonian operator applied to the wavefunction.
def hamiltonian(x, alpha):
    # Compute the second derivative (discretized)
    psi = wavefunction(x, alpha)
    d2dx2 = (np.roll(psi, -1) - 2 * psi + np.roll(psi, 1)) / dx**2
    
    # Kinetic energy term
    T = -0.5 * d2dx2
    
    # Potential energy temr
    V = 0.5 * m * (omega**2) * (x**2) * psi
    
    return (T + V)

# Compute the expectation value of the Hamiltonian.
def expectation_value(x, alpha):
    psi  = wavefunction(x, alpha)
    Hpsi = hamiltonian (x, alpha)
    
    # Numerically integrate the expectation value (normalization included)
    numerator = np.sum(np.conj(psi) * Hpsi) * dx
    denominator = np.sum(np.abs(psi)**2) * dx
    
    return (numerator/denominator)

# Compute the numerical gradient of the expectation value w.r.t. alpha.
def gradient(x, alpha, epsilon=1.0e-6):
    E_plus  = expectation_value(x, alpha + epsilon)
    E_minus = expectation_value(x, alpha - epsilon)
    return (E_plus - E_minus) / (2 * epsilon)

# Perform gradient descent to minimize the expectation value and find the ground state.
def gradient_descent(x, alpha_init, learning_rate=0.01, iterations=100):
    alpha = alpha_init
    alpha_history  = [alpha]
    energy_history = []
    
    for iter in range(iterations):
        energy = expectation_value(x, alpha)
        energy_history.append(energy)
        
        # Calculate the gradient and update alpha
        grad   = gradient(x, alpha)
        alpha -= learning_rate * grad
        
        alpha_history.append(alpha)
    
    return alpha, energy_history, alpha_history

def plot_hamilton_results(energy_history, alpha_history):
    plt.figure(figsize=(6, 3)) 
    
    # Plot energy history
    plt.subplot(1, 2, 1)
    plt.plot(energy_history, label="Energy History", lw=1, color="royalblue")
    plt.xlabel("Iteration")
    plt.ylabel("Energy")
    plt.title("Energy Convergence")
    plt.grid(True)
    
    # Plot alpha history
    plt.subplot(1, 2, 2)
    plt.plot(alpha_history, label="Alpha History", lw=1, color="forestgreen")
    plt.xlabel("Iteration")
    plt.ylabel("Alpha")
    plt.title("Alpha Convergence")
    plt.grid(True)
    
    plt.show()

# Main code to find the ground state
alpha_init    = 1.0  # Initial guess for alpha
learning_rate = 0.05
iterations    = 200  # Number of iterations for gradient descent

# Perform gradient descent to minimize the energy
alpha_opt, energy_history, alpha_history = gradient_descent(x, alpha_init, learning_rate, iterations)

# optimized variational parameter and the final energy
print(f"Optimized alpha: {alpha_opt}")
print(f"Final energy: {energy_history[-1]}")

# Plot the results
plot_hamilton_results(energy_history, alpha_history)


***
# 📈 Solution
For the data stored in `../datasets/simple_grad_descent_data1.csv`, I used $f(x)=3x^4-6x^3-2$ and for the data stored in `../datasets/simple_grad_descent_data2.csv`, $f(x)=5/((x-1.5)^2+1)$.

***
END
***